## Домашнее задание



### 1. Локализация корней.

Локализовать действительные корни в уравнении:

$$
f(x)=20 x^{3} - 4 x^{2} - 5 x + 1 .
$$

In [ ]:
import numpy as np
import sympy as smp

In [ ]:
# Взято из __Пример. Нахождение всех корней многочлена.__
# A list of colors to distinguish the roots.
colors = ['xkcd:eggshell', 'mediumseagreen', 'c', 'xkcd:sky blue']

TOL = 1.e-8

def newton(z0, f, fprime, MAX_IT=1000):
    """The Newton-Raphson method applied to f(z).

    Returns the root found, starting with an initial guess, z0, or False
    if no convergence to tolerance TOL was reached within MAX_IT iterations.

    """

    z = z0
    for i in range(MAX_IT):

        # Вместо всей этой секции z = optimize.newton(f, z0, fprime)

        dz = f(z)/fprime(z)
        if abs(dz) < TOL:
            return z
        z -= dz
    return False

def plot_newton_fractal(f, fprime, n=200, domain=(-1, 1, -1, 1)):
    """Plot a Newton Fractal by finding the roots of f(z).

    The domain used for the fractal image is the region of the complex plane
    (xmin, xmax, ymin, ymax) where z = x + iy, discretized into n values along
    each axis.

    """

    roots = []
    m = np.zeros((n, n))

    def get_root_index(roots, r):
        """Get the index of r in the list roots.

        If r is not in roots, append it to the list.

        """

        try:
            return np.where(np.isclose(roots, r, atol=TOL))[0][0]
        except IndexError:
            roots.append(r)
            return len(roots) - 1

    xmin, xmax, ymin, ymax = domain
    for ix, x in enumerate(np.linspace(xmin, xmax, n)):
        for iy, y in enumerate(np.linspace(ymin, ymax, n)):
            z0 = x + y*1j
            r = newton(z0, f, fprime)
            if r is not False:
                ir = get_root_index(roots, r)
                m[iy, ix] = ir


    count_r_a = [0, 0]
    max_ = -1
    i_max = -1

    for i, root in enumerate(roots):
        abs_ = root.real**2 + root.imag**2

        if abs_ > max_:
            max_ = abs_
            i_max = i

    for ix, x in enumerate(np.linspace(xmin, xmax, n)): # Считаем долю площади наиб. корня
        for iy, y in enumerate(np.linspace(ymin, ymax, n)):
            if x**2 + y**2 < 10**2:
                count_r_a[1] += 1

                if (m[iy, ix] == i_max):
                    count_r_a[0] += 1

    nroots = len(roots)
    if nroots > len(colors):
        # Use a "continuous" colormap if there are too many roots.
        cmap = 'hsv'
    else:
        # Use a list of colors for the colormap: one for each root.
        cmap = ListedColormap(colors[:nroots])

    plt.figure(figsize=(7,7))
    plt.imshow(m, extent = [xmin, xmax, ymin, ymax], cmap=cmap)

    for i, root in enumerate(roots):
        plt.scatter(root.real, root.imag)

    plt.show()

    return roots, float(count_r_a[0]/count_r_a[1])*np.pi*100

f = lambda z: 20*z**3 - 4*z**2 - 5*z + 1
fprime = lambda z: 60*z**2 -8*z - 5

domain=(-10, 10, -10, 10)
n = 400

roots, m = plot_newton_fractal(f, fprime, n=n, domain=domain)
print(roots)
print(f"S = {m}")

### 2. Порядок сходимости итерационного метода.

Определить порядок сходимости итерационного метода при вычислении квадратного корня $x^* = \sqrt a$ :



$$
x_{n+1}=x_n - \frac{11x_n^4 - 4x_n^2 a + a^2}{16 x_n^5} (x_n^2 - a)
$$

In [26]:
from sympy import symbols, sqrt, diff

x, a = symbols('x a')

def F(x):
    return x - (11 * (x ** 4) - 4 * a * (x ** 2) + (a ** 2)) * (x ** 2 - a) / (16 * (x ** 5))

def F1(x):
    return 1 - (a**2 - 4*a*x**2 + 11*x**4)/(8*x**4) - (-a + x**2)*(-8*a*x + 44*x**3)/(16*x**5) + 5*(-a + x**2)*(a**2 - 4*a*x**2 + 11*x**4)/(16*x**6)

# Производная по x
derivative_F = diff(F(x), x)

# Подставим \sqrt(a) 
result_derivative_at_sqrt_a = derivative_F.subs(x, sqrt(a)) # в точке \sqrt(a) производная = 0

print(result_derivative_at_sqrt_a)


0


In [29]:
def F2(x):
    return (2*a - 11*x**2 - (a - x**2)*(2*a - 33*x**2)/(4*x**2) + 5*(a - x**2)*(2*a - 11*x**2)/(2*x**2) + 9*(a**2 - 4*a*x**2 + 11*x**4)/(8*x**2) + 15*(a - x**2)*(a**2 - 4*a*x**2 + 11*x**4)/(8*x**4))/x**3
print(F2(sqrt(a))) # третья производная тоже =0 в \sqrt(a)

diff(F(x), x, 2)

0


(2*a - 11*x**2 - (a - x**2)*(2*a - 33*x**2)/(4*x**2) + 5*(a - x**2)*(2*a - 11*x**2)/(2*x**2) + 9*(a**2 - 4*a*x**2 + 11*x**4)/(8*x**2) + 15*(a - x**2)*(a**2 - 4*a*x**2 + 11*x**4)/(8*x**4))/x**3

In [31]:
def F3(x):
    return 3*(-20*a + 220*x**2 + 10*(a - x**2)*(2*a - 33*x**2)/x**2 - 60*(a - x**2)*(2*a - 11*x**2)/x**2 - 25*(a**2 - 4*a*x**2 + 11*x**4)/x**2 - 35*(a - x**2)*(a**2 - 4*a*x**2 + 11*x**4)/x**4)/(8*x**4)

print(F3(sqrt(a))) # третья производная тоже =0 в sqrt(a)

diff(F(x), x, 3)  

0


3*(-20*a + 220*x**2 + 10*(a - x**2)*(2*a - 33*x**2)/x**2 - 60*(a - x**2)*(2*a - 11*x**2)/x**2 - 25*(a**2 - 4*a*x**2 + 11*x**4)/x**2 - 35*(a - x**2)*(a**2 - 4*a*x**2 + 11*x**4)/x**4)/(8*x**4)

In [32]:
def F4(x):
    return 3*(-44 - 209*(a - x**2)/(2*x**2) - 9*(2*a - 33*x**2)/x**2 + 50*(2*a - 11*x**2)/x**2 - 15*(a - x**2)*(2*a - 33*x**2)/x**4 + 70*(a - x**2)*(2*a - 11*x**2)/x**4 + 55*(a**2 - 4*a*x**2 + 11*x**4)/(2*x**4) + 35*(a - x**2)*(a**2 - 4*a*x**2 + 11*x**4)/x**6)/x**3

print(F4(sqrt(a))) 

diff(F(x), x, 4)  

15/a**(3/2)


3*(-44 - 209*(a - x**2)/(2*x**2) - 9*(2*a - 33*x**2)/x**2 + 50*(2*a - 11*x**2)/x**2 - 15*(a - x**2)*(2*a - 33*x**2)/x**4 + 70*(a - x**2)*(2*a - 11*x**2)/x**4 + 55*(a**2 - 4*a*x**2 + 11*x**4)/(2*x**4) + 35*(a - x**2)*(a**2 - 4*a*x**2 + 11*x**4)/x**6)/x**3

Таким образом, включительно до 3 порядка производные равны нулю в искомой точке, а четвертая не равна. Следовательно порядок сходимости равен 3



### 3. Метод Ньютона и Гаусса-Ньютона.

Решение проблемы многомерной линейной регрессии нормальным уравнением очень похоже на обобщение метода Ньютона на многомерный случай. Но это не так.
Укажите, в чём различие между методами. В одномерном случае

$$
f(x) \approx f(x_0) + f^{\prime}(x_0)(x - x_0) = 0
$$

$$
x =  x_0 - \frac{f(x_0)}{f^{\prime}(x_0)}
$$

Обобщение метода Ньютона на многомерный случай выглядит так.

$$
F(\vec{β}) \approx F(\vec{β^{(0)}}) + \sum\limits_{i=1}^n{\frac{∂F(\vec{β^{(0)}})}{∂β_i}(β_i - β^0_i)} = 0
$$

$$
F(\vec{β^{(0)}}) + \nabla F(\vec{β^{(0)}})\vec{p}   = 0
$$

В случае поиска минимума функции F к нулю приравниваем частные производные F
$$
\frac{∂F(\vec{β})}{∂β_i} \approx \frac{∂F(\vec{β^{(0)}})}{∂β_i} + \sum\limits_{j=1}^n{\frac{∂^2F(\vec{β^{(0)}})}{∂β_i∂β_j}(β_i - β^0_i)} = 0
$$

Хинт: покажите, что

$$
2J^TJ  \neq   H_{ij}
$$

$H_{ij}$ - гессиан F.












РЕШЕНИЕ ТУТ

В случае многомерной линейной регрессии имеется в виду уравнение:
$$ w = (X^T X)^{-1} X^Ty$$
Которое является точным решением для поиска минимума функции потерь $|y - Xw|^2_2$, то есть это не иттерационным метод, и сравнение непонятно изначально


По поводу неравенства матриц, сложно понять, что имелось в виду под этим заданием, так как если F - скалярная функция, то J - вектор, следовательно $J^T J$ - число, а не матрица, а если F - вектор функция, то H - тензор третьего ранга и получить его перемножением квадратных матриц не получится.

### 4. Зри в корень

Отделить корни следующих уравнений, а затем уточнить один из них с помощью подходящего итерационно процесса (любых двух на ваш выбор двумя разными методами):


a) $(0.5)^x+1=(x-1)^2$,

__b)__ $(x-3) \cos x=1, \quad-2 \pi \leq \mathrm{x} \leq 2 \pi$,

c) $\operatorname{arctg}(x-1)+2 x=0$,

__d)__ $x^2-20 \sin x=0$

e) $2 \operatorname{tg} x-x / 2+1=0$,

__f)__ $2 \lg x-x / 2+1=0$,

g) $x^2-e^x / 5=0$

__h)__ $\ln x+(x-1)^3=0$,

i) $x 2^x=1$

__j)__ $(x+1)^{0.5}=1 / x$.

In [26]:
import numpy as np

def Fg(x):
    return x ** 2 - np.exp(x) / 5

def Jg(x):
    return 2 * x - np.exp(x) / 5

def Fi(x):
    return x * (2 ** x) - 1

def Ji(x):
    return 2 ** x + x * (2 ** x) * np.log(2)

def Newton_system(F, J, x, eps):
    """
    Solve nonlinear system F=0 by Newton's method.
    J is the Jacobian of F. Both F and J must be functions of x.
    At input, x holds the start value. The iteration continues
    until ||F|| < eps.
    """
    F_value = F(x)
    F_norm = abs(F_value)  # l2 norm of vector
    iteration_counter = 0
    while F_norm > eps and iteration_counter < 100:
        delta = - F(x) / J(x)
        x = x + delta
        F_value = F(x)
        F_norm = abs(F_value)
        iteration_counter += 1

    # Here, either a solution is found, or too many iterations
    if F_norm > eps:
        iteration_counter = -1
    return x, iteration_counter

def Secant(F, x=4, eps=1e-9):
    x_prev = x
    x_cur = x_prev + 0.1
    F_value = F(x_cur)
    F_norm = abs(F_value)  # l2 norm of vector
    iteration_counter = 0
    while F_norm > eps and iteration_counter < 100:
        delta = - F(x_cur) * (x_cur - x_prev) / (F(x_cur) - F(x_prev))
        x_prev = x_cur
        x_cur = x_cur + delta
        F_norm = abs(F(x_cur))
        iteration_counter += 1

    # Here, either a solution is found, or too many iterations
    if F_norm > eps:
        iteration_counter = -1
    return x_cur, iteration_counter

In [27]:
Newton_system(Fg, Jg, x=4, eps=1e-9)

(4.7079379181288585, 7)

In [28]:
Secant(Fg, x=4, eps=1e-9)

(4.7079379181288585, 9)

In [13]:
Newton_system(Fi, Ji, x=0.5, eps=1e-9)

(0.641185744504986, 4)

In [29]:
Secant(Fi, x=0.5, eps=1e-9)

(0.6411857445143642, 4)

### 5. Зри в корень дважды

Вычислить с точностью $\varepsilon=10^{-3}$ координаты точек пересечения кривых (любых двух на ваш выбор двумя разными методами):

a)
$$
\left\{\begin{array}{l}
\sin (x+1)-y=1.2 \\
2 x+\cos (y)=2
\end{array}\right.
$$
б)
$$
\left\{\begin{array}{l}
\tan (x y+0.4)=x^2 \\
0.6 x^2+2 y^2=1
\end{array}\right.
$$
B)
$$
\left\{\begin{array}{l}
\cos (x-1)+y=0.5 \\
x-\cos (y)=3
\end{array}\right.
$$
г)
$$
\left\{\begin{array}{l}
\sin (x+2)-y=1.5 \\
x+\cos (y-2)=0.5
\end{array}\right.
$$

In [30]:
import numpy as np


def Newton_system(F, J, x, eps):
    """
    Solve nonlinear system F=0 by Newton's method.
    J is the Jacobian of F. Both F and J must be functions of x.
    At input, x holds the start value. The iteration continues
    until ||F|| < eps.
    """
    F_value = F(x)
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    iteration_counter = 0
    while abs(F_norm) > eps and iteration_counter < 100:
        delta = np.linalg.solve(J(x), -F_value)
        x = x + delta
        F_value = F(x)
        F_norm = np.linalg.norm(F_value, ord=2)
        iteration_counter += 1

    # Here, either a solution is found, or too many iterations
    if abs(F_norm) > eps:
        iteration_counter = -1
    return x, iteration_counter


def Jacobi_system(F, Fi, x, eps):
    """
    Solve nonlinear system F=0 by Jacobi's method.
    At input, x holds the start value. The iteration continues
    until ||F|| < eps.
    x = Fi(x)
    for a)
    """
    F_value = F(x)
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    iteration_counter = 0
    while abs(F_norm) > eps and iteration_counter < 100:
        x = Fi(x)
        F_value = F(x)
        F_norm = np.linalg.norm(F_value, ord=2)
        iteration_counter += 1

    # Here, either a solution is found, or too many iterations
    if abs(F_norm) > eps:
        iteration_counter = -1
    return x, iteration_counter

In [33]:
def Fi_a(x):
    return np.array([1 - 0.5*np.cos(x[1]), np.sin(x[0] + 1) - 1.2])

def F_a(x):
    return np.array([np.sin(x[0] + 1) - x[1] - 1.2, 2*x[0] + np.cos(x[1]) - 2])

def F_c(x):
    return np.array([np.cos(x[0] - 1) + x[1] - 0.5, x[0] - np.cos(x[1]) - 3])

def J_c(x):
    return np.array([[-np.sin(x[0] - 1), 1], [1, np.sin(x[1])]])

In [34]:
Jacobi_system(F_a, Fi_a, x=np.array([1, -0.5]), eps=1e-6)

(array([ 0.51015052, -0.20183835]), 6)

In [37]:
Newton_system(F_c, J_c, x=np.array([1, 1]), eps=1e-6)

(array([3.35591174, 1.20690682]), 10)

### 6*. Оценка скорости сходимости метода Ньютона.
Покажите, что для функции $f(x)=|x|^{5 / 2}$ метод Ньютона сходится лишь экспонциально - т.е. невязка уменьшается пропорционально $e^{-n}$.

Покажите аналитически, что метод Ньютона в лучшем случае имеет квадратичную экспонциальную сходимость, т.е. ошибка убывает пропорционально $e^{-n^2}$.